In [ ]:
from sympy import randprime, mod_inverse
import matplotlib.pyplot as plt
from statistics import mean
import random

plt.rcParams["figure.figsize"] = (20, 9)


In [ ]:
def GenModulus(w):
    n = len(w) // 2
    p = randprime(2**n, 2 ** (n + 1))
    q = randprime(2**n, 2 ** (n + 1))
    N = p * q
    return N, p, q


In [ ]:
def GenRSA(w):
    N, p, q = GenModulus(w)
    m = (p - 1) * (q - 1)
    e = 2**16 + 1
    d = mod_inverse(e, m)
    return N, e, d, p, q


In [ ]:
def enc(m, N, e):
    return fast_pow(m, N, e)  # x ** e % N


def dec(c, N, d):
    return fast_pow(c, N, d)  # c ** d % N


In [ ]:
def fast_pow(c, N, exp):
    exp_bin = "{0:b}".format(exp)
    # print(exp_bin)
    exp_len = len(exp_bin)
    reductions = 0
    h = 0
    x = c
    for j in range(1, exp_len):
        x, r = mod_reduce(x**2, N)
        reductions = reductions + r
        if exp_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reductions = reductions + r
            h = h + 1
    return x, h, reductions


In [ ]:
def mod_reduce(a, b):
    reductions = 0
    if a >= b:
        a = a % b
        reductions = 1
    return a, reductions


In [ ]:
def gen_bool_range_with_exponents_in(N, c_range, smaller_exp, bigger_exp):
    c_less_then_bigger_bool = []
    c_between_bool = []
    for c in c_range:
        c_less_then_bigger_bool.append(c**bigger_exp < N)
        c_between_bool.append(c**smaller_exp < N < c**bigger_exp)
    return c_less_then_bigger_bool, c_between_bool

def gen_range_with_exponents_in(N, c_range, smaller_exp, bigger_exp):
    c_less_then_bigger = []
    c_between = []
    for c in c_range:
        if c**bigger_exp < N:
            c_less_then_bigger.append(c)
        if c**smaller_exp < N < c**bigger_exp:
            c_between.append(c)
    return c_less_then_bigger, c_between

In [ ]:
w = "111111111111111111111111111111111111"
N, e, d, p, q = GenRSA(w)
print(f"{N=} {e=} {d=} {p=} {q=}")
print("{0:b}".format(d))
c_range = range(3, 800000)


In [ ]:
r_array = []
for c in c_range:
    decrypted = dec(c, N, d)
    # print(f"{c=}, {decrypted[2]}")
    r_array.append(decrypted[2])


In [ ]:
c_less_then_bigger_bool, c_between_bool = gen_bool_range_with_exponents_in(
    N=N, c_range=c_range, smaller_exp=2, bigger_exp=3
)
plt.plot(
    c_range,
    c_less_then_bigger_bool,
    "rs",
    c_range,
    c_between_bool,
    "g^",
    c_range,
    r_array
)


In [ ]:
def ask_for_decryption(smaller_exp, bigger_exp, c_range):
    c_less_then_bigger, c_between = gen_range_with_exponents_in(
        N=N, c_range=c_range, smaller_exp=smaller_exp, bigger_exp=bigger_exp
    )

    c_less_then_bigger_array = []
    for c in c_less_then_bigger:
        decrypted = dec(c, N, d)
        c_less_then_bigger_array.append(decrypted[2])

    c_between_array = []
    for c in c_between:
        decrypted = dec(c, N, d)
        c_between_array.append(decrypted[2])

    print(c_less_then_bigger_array[:10], c_between_array[:10])
    print(f"{smaller_exp=}, {bigger_exp=}")
    print(mean(c_less_then_bigger_array), mean(c_between_array))

    e = 1
    if mean(c_less_then_bigger_array) + e < mean(c_between_array):
        return "1"
    else:
        return "0"


def timing_attack(d_length, c_range):
    guessed_d = ""
    for exp in range(d_length):
        bit = ask_for_decryption(
            smaller_exp=3 * exp + 2, bigger_exp=3 * exp + 3, c_range=c_range
        )
        print(bit)
        guessed_d += bit

    print(guessed_d)


In [ ]:
n = len(w) // 2
print("{0:b}".format(d))
timing_attack(d_length=n, c_range=c_range)


In [ ]:
def my_fast_pow(c, N, exp_bin):
    exp_len = len(exp_bin)
    reductions_list = [-1 for _ in range(2 * exp_len)]
    x = c
    for j in range(1, exp_len):
        x, r = mod_reduce(x**2, N)
        reductions_list[2*j] = r
        # print(f'{exp_bin[j]=}')
        if exp_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reductions_list[2*j + 1] = r
    return reductions_list

def find_lists_differ_last_bit(d):
    pairs = []
    for key, value in d.items():
        for key_2, value_2 in d.items():
            # print(f'{value[:-1]=} {value_2[:-1]=} {value[-1]=} {value_2[-1]=}')
            if value[:-1] == value_2[:-1]:
                # print(value[-1], value_2[-1])
                    # and value[-1] != value_2[-1]:
                pairs.append([key, value, key_2, value_2])
    return pairs

def find_message_with_given_prefix(match, N, exp_bin):
    while True:
        c = random.choice(c_range)
        reductions = my_fast_pow(c, N, exp_bin)
        if reductions[:-1] == match[:-1]:
            print(reductions, match)


In [66]:
tries_number = 200000
mes_red = {}
y_tries, x_tries = [], []
exp_bin = "1111"
i = 0
while i < tries_number:
    i += 1
    c = random.choice(c_range)
    reductions = my_fast_pow(c, N, exp_bin)
    mes_red[c] = reductions
    if reductions[-1] == 0:
        print(reductions)
        # print(reductions[:-1])

# pairs = find_lists_differ_last_bit(mes_red)
# print(mes_red)
# print(pairs)


[-1, -1, 0, 1, 1, 1, 1, 0]


KeyboardInterrupt: 

In [72]:
find_message_with_given_prefix([-1, -1, 0, 1, 1, 1, 1, 0], N, exp_bin)
